In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import anndata
import scanpy as sc
sc.settings.n_jobs = 56
sc.settings.set_figure_params(dpi=180, dpi_save=300, frameon=False, figsize=(4, 4), fontsize=8, facecolor='white')

import sys
sys.path.append('/home/xingjiepan/Softwares/ZhuangLab/MERFISH_analysis/cell_cell_contact')
from permutation import generate_cell_type_contact_count_matrices

In [2]:
# Make the output path
output_path = 'outputs_15um'
os.makedirs(output_path, exist_ok=True)

# Count cell-cell contacts at the subclass level

In [3]:
%%time
major_brain_regions = ['Cerebellum', 'Cortical_subplate', 'Fiber_tracts', 'Hippocampus',
       'Isocortex', 'Medulla', 'Olfactory', 'Pallidum', 'Pons', 'Striatum',
       'Thalamus', 'Ventricular_systems', 'anterior_HY', 'anterior_MB',
       'posterior_HY', 'posterior_MB']

for region in major_brain_regions:
    print(region)
    
    # Read the data
    df_ct_labels = pd.read_csv(os.path.join('cells_by_regions', f'{region}.csv'), index_col=0)
    slice_ids = np.unique(df_ct_labels['slice_id'])
    
    cell_type_col = 'subclass_label_transfer'
    cell_types = np.unique(df_ct_labels[cell_type_col])
    N_cell_types = len(cell_types)
    N_permutations = 1000
    
    # Count and save the contacts without permutation
    merged_contact_counts = np.zeros((N_cell_types, N_cell_types), dtype=int)

    for slice_id in slice_ids:
        print(f'Count for {slice_id}')
        df_slice = df_ct_labels[df_ct_labels['slice_id'] == slice_id]
        cell_type_contact_counts = generate_cell_type_contact_count_matrices(df_slice, cell_type_col, 
                                        ['center_x', 'center_y'], cell_types, 
                                        permutation_method='no_permutation', contact_radius=15)
    
        merged_contact_counts = merged_contact_counts + cell_type_contact_counts
    
    output_file = os.path.join(output_path, f'{region}_no_permutation.npy')
    np.save(output_file, merged_contact_counts)
    
    from multiprocessing import Pool

    def permute_and_count_contacts_for_slices(df_slice_list):
        merged_contact_counts = np.zeros((N_permutations, N_cell_types, N_cell_types), dtype=int)
        for df_slice in df_slice_list:
            
            for i in range(N_permutations):
                df_slice_rand = df_slice.copy()
                
                r_permute = 100
                r = r_permute * np.sqrt(np.random.uniform(size=df_slice_rand.shape[0]))
                theta = np.random.uniform(size=df_slice_rand.shape[0]) * 2 * np.pi
                
                df_slice_rand['center_x'] += r * np.sin(theta)
                df_slice_rand['center_y'] += r * np.cos(theta)
            
                cell_type_contact_counts = generate_cell_type_contact_count_matrices(df_slice_rand, cell_type_col, 
                                        ['center_x', 'center_y'], cell_types, 
                                        permutation_method='no_permutation', contact_radius=15)
            
            
                merged_contact_counts[i] = merged_contact_counts[i] + cell_type_contact_counts
        
        return merged_contact_counts
    
    # Get the dataframe for each slice
    all_df_slice_list = [df_ct_labels[df_ct_labels['slice_id'] == slice_id] for slice_id in slice_ids]

    # Split the slices into groups
    N_groups = 48
    group_size = int(np.ceil(len(slice_ids) / N_groups))
    grouped_slice_dfs = []

    for i in range(N_groups):
        slice_id_start = i * group_size
        slice_id_stop = (i + 1) * group_size
        grouped_slice_dfs.append(all_df_slice_list[slice_id_start:slice_id_stop])
    
    # Permute and count the contacts in parallel
    N_permutations = 1000
    with Pool(N_groups) as p:
        contact_analysis_results = p.map(permute_and_count_contacts_for_slices, grouped_slice_dfs)
    
    merged_contact_counts = np.sum(contact_analysis_results, axis=0)
    
    means = np.mean(merged_contact_counts, axis=0)
    stds = np.std(merged_contact_counts, axis=0)
    
    np.save(os.path.join(output_path, f'{region}_local_permutation_count_tensor.npy'),
            merged_contact_counts)
    
    output_file_mean = os.path.join(output_path, f'{region}_local_permutation_mean.npy')
    np.save(output_file_mean, means)
    output_file_std = os.path.join(output_path, f'{region}_local_permutation_std.npy')
    np.save(output_file_std, stds)

    #break

Cerebellum
Count for co1_slice48
Count for co1_slice49
Count for co1_slice50
Count for co1_slice51
Count for co1_slice52
Count for co1_slice53
Count for co1_slice54
Count for co1_slice55
Count for co1_slice56
Count for co1_slice57
Count for co1_slice58
Count for co1_slice59
Count for co1_slice60
Count for co1_slice61
Count for co2_slice100
Count for co2_slice102
Count for co2_slice103
Count for co2_slice104
Count for co2_slice105
Count for co2_slice106
Count for co2_slice107
Count for co2_slice108
Count for co2_slice109
Count for co2_slice110
Count for co2_slice111
Count for co2_slice112
Count for co2_slice113
Count for co2_slice114
Count for co2_slice115
Count for co2_slice116
Count for co2_slice117
Count for co2_slice118
Count for co2_slice119
Count for co2_slice120
Count for co2_slice121
Count for co2_slice122
Count for co2_slice123
Count for co2_slice124
Count for co2_slice125
Count for co2_slice126
Count for co2_slice127
Count for co2_slice128
Count for co2_slice129
Count for co2_

Count for sa1_slice1
Count for sa1_slice10
Count for sa1_slice11
Count for sa1_slice12
Count for sa1_slice13
Count for sa1_slice14
Count for sa1_slice15
Count for sa1_slice16
Count for sa1_slice17
Count for sa1_slice18
Count for sa1_slice19
Count for sa1_slice2
Count for sa1_slice20
Count for sa1_slice21
Count for sa1_slice22
Count for sa1_slice3
Count for sa1_slice4
Count for sa1_slice5
Count for sa1_slice6
Count for sa1_slice7
Count for sa1_slice8
Count for sa1_slice9
Count for sa2_slice1
Count for sa2_slice2
Count for sa2_slice3
Hippocampus
Count for co1_slice25
Count for co1_slice26
Count for co1_slice27
Count for co1_slice28
Count for co1_slice30
Count for co1_slice31
Count for co1_slice32
Count for co1_slice33
Count for co1_slice34
Count for co1_slice35
Count for co1_slice36
Count for co1_slice37
Count for co1_slice39
Count for co1_slice40
Count for co1_slice41
Count for co1_slice42
Count for co1_slice44
Count for co1_slice45
Count for co1_slice46
Count for co1_slice47
Count for 

Count for co1_slice18
Count for co1_slice19
Count for co1_slice20
Count for co1_slice21
Count for co1_slice22
Count for co1_slice23
Count for co1_slice25
Count for co1_slice26
Count for co1_slice27
Count for co1_slice28
Count for co1_slice30
Count for co1_slice31
Count for co1_slice32
Count for co1_slice33
Count for co1_slice34
Count for co1_slice35
Count for co1_slice36
Count for co1_slice37
Count for co1_slice39
Count for co1_slice4
Count for co1_slice40
Count for co1_slice41
Count for co1_slice42
Count for co1_slice44
Count for co1_slice45
Count for co1_slice46
Count for co1_slice47
Count for co1_slice48
Count for co1_slice5
Count for co1_slice6
Count for co1_slice7
Count for co1_slice8
Count for co1_slice9
Count for co2_slice10
Count for co2_slice100
Count for co2_slice101
Count for co2_slice102
Count for co2_slice103
Count for co2_slice11
Count for co2_slice12
Count for co2_slice13
Count for co2_slice14
Count for co2_slice15
Count for co2_slice16
Count for co2_slice17
Count for co

Count for co1_slice31
Count for co1_slice32
Count for co1_slice33
Count for co1_slice34
Count for co1_slice35
Count for co1_slice36
Count for co1_slice37
Count for co1_slice39
Count for co1_slice40
Count for co1_slice41
Count for co1_slice42
Count for co1_slice44
Count for co1_slice45
Count for co1_slice46
Count for co1_slice47
Count for co1_slice48
Count for co2_slice100
Count for co2_slice101
Count for co2_slice102
Count for co2_slice103
Count for co2_slice64
Count for co2_slice65
Count for co2_slice66
Count for co2_slice67
Count for co2_slice68
Count for co2_slice69
Count for co2_slice70
Count for co2_slice71
Count for co2_slice72
Count for co2_slice73
Count for co2_slice74
Count for co2_slice75
Count for co2_slice76
Count for co2_slice77
Count for co2_slice78
Count for co2_slice79
Count for co2_slice80
Count for co2_slice81
Count for co2_slice82
Count for co2_slice83
Count for co2_slice84
Count for co2_slice85
Count for co2_slice86
Count for co2_slice87
Count for co2_slice88
Count 

Count for co2_slice122
Count for co2_slice123
Count for co2_slice124
Count for co2_slice125
Count for sa1_slice1
Count for sa1_slice10
Count for sa1_slice2
Count for sa1_slice3
Count for sa1_slice4
Count for sa1_slice5
Count for sa1_slice6
Count for sa1_slice7
Count for sa1_slice8
Count for sa1_slice9
CPU times: user 2min, sys: 27.7 s, total: 2min 27s
Wall time: 26min 8s
